In [3]:
import cv2
import numpy as np

# Set the fixed size of the output video
OUTPUT_SIZE = (640, 480)

# Open the input video file
cap = cv2.VideoCapture('C:/Users/nihal/Desktop/Academics/Main/MainProject/Main_Project/Project/videos/Crowd_Video.mp4')

# Create a VideoWriter object to write the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('C:/Users/nihal/Desktop/Academics/Main/MainProject/Main_Project/Project/videos/output_video.mp4', fourcc, 25.0, OUTPUT_SIZE)

# Loop through each frame of the input video
while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        # Resize the frame to the fixed output size
        resized_frame = cv2.resize(frame, OUTPUT_SIZE)
        
        # Normalize the pixel values to between 0 and 1
        normalized_frame = resized_frame.astype(np.float32) / 255.0
        
        # Write the normalized frame to the output video
        out.write(normalized_frame)
        
        cv2.imshow('C:/Users/nihal/Desktop/Academics/Main/MainProject/Main_Project/Project/frame1/Resized and Normalized Frame', normalized_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release the resources
cap.release()
out.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input

# Load the pre-trained VGG16 model
model = VGG16(weights='imagenet', include_top=False)

# Set the fixed size of the input frames
INPUT_SIZE = (224, 224)

# Open the input video file
cap = cv2.VideoCapture('C:/Users/nihal/Desktop/Academics/Main/MainProject/Main_Project/Project/videos/Crowd_Video.mp4')

# Create an empty list to store the extracted features
features = []

# Loop through each frame of the input video
while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        # Resize the frame to the fixed input size
        resized_frame = cv2.resize(frame, INPUT_SIZE)
        
        # Preprocess the input frame for the VGG16 model
        preprocessed_frame = preprocess_input(np.expand_dims(resized_frame, axis=0))
        
        # Extract the features from the preprocessed frame using the VGG16 model
        features.append(model.predict(preprocessed_frame)[0])
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release the resources
cap.release()

# Convert the extracted features to a NumPy array
features = np.array(features)


1/1 [==============================] - 0s 111ms/step


In [86]:
print(features)

[[[[0.00000000e+00 1.29071398e+01 0.00000000e+00 ... 0.00000000e+00
    2.07962108e+00 0.00000000e+00]
   [0.00000000e+00 6.64372396e+00 0.00000000e+00 ... 0.00000000e+00
    3.92798781e+00 0.00000000e+00]
   [8.38500500e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    6.92231834e-01 0.00000000e+00]
   ...
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    4.66093874e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    1.30598402e+01 0.00000000e+00]]

  [[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    4.89280653e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   ...
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00

In [79]:

np.save('C:/Users/nihal/Desktop/Academics/Main/MainProject/Main_Project/Project/features.npy',features)
videoclass="MBA Crowd"
feature_array=[]
feature_array.append([features,videoclass])
# print(feature_array)
import numpy as np
import pandas as pd
# fea=np.mean(feature_array.T,axis=0)
# print(feature_array)
ee=pd.DataFrame(feature_array,columns=['f','c'])
print(ee)
ee.to_csv('C:/Users/nihal/Desktop/Academics/Main/MainProject/Main_Project/Project/my_data.csv', index=False)

                                                   f          c
0  [[[[ 0.         12.90714     0.          0.   ...  MBA Crowd


In [129]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
dataset = np.array(features)

# Split the dataset into training and validation sets
train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Extract the shape of the feature map
feature_shape = train_data.shape[1:-1]

print(f"Feature map shape: {feature_shape}")


Feature map shape: (7, 7)


In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Multiply, GlobalAveragePooling2D, Reshape, Dense, Add
from tensorflow.keras.models import Model

# Define the input shape
input_shape = (7, 7, 1)  # replace with the actual shape of your feature map

# Define the attention mechanism
def attention_block(x):
    # Channel attention
    c_avg = GlobalAveragePooling2D()(x)
    c_dense_1 = Dense(units=int(x.shape[-1]), activation='relu')(c_avg)
    c_dense_2 = Dense(units=int(x.shape[-1]), activation='sigmoid')(c_dense_1)
    c_out = Multiply()([x, c_dense_2])
    
    # Spatial attention
    s_conv = Conv2D(filters=1, kernel_size=3, padding='same', activation='sigmoid')(x)
    s_out = Multiply()([x, s_conv])
    
    # Hybrid attention
    out = Add()([c_out, s_out])
    
    return out

# Define the model architecture
inputs = Input(shape=input_shape)
x = Conv2D(filters=512, kernel_size=3, padding='same')(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = attention_block(x)
x = Conv2D(filters=1, kernel_size=1)(x)
x = Activation('relu')(x)
x = Reshape((-1,))(x)
model = Model(inputs=inputs, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Specify the path where you want to save the model
path = 'C:/Users/nihal/Desktop/Academics/Main/MainProject/Main_Project/Project/model.h5'

# Save the model to the specified path in HDF5 format
model.save(path, save_format='h5')

In [7]:
import cv2
import numpy as np
import tensorflow as tf

# Define the paths to the trained model and the video file
model_path = 'C:/Users/nihal/Desktop/Academics/Main/MainProject/Main_Project/Project/model.h5'
video_path = 'C:/Users/nihal/Desktop/Academics/Main/MainProject/Main_Project/Project/videos/Crowd_Video.mp4'

# Load the trained model
model = tf.keras.models.load_model(model_path)

# Define the input shape of the model
input_shape = (7,7,1)  # replace with the actual shape of your feature map

# Open the video file
cap = cv2.VideoCapture(video_path)

# Initialize the headcount
headcount = 0

# Loop over the frames in the video
while cap.isOpened():
    # Read the next frame
    ret, frame = cap.read()
    
    # If the frame is not valid, break the loop
    if not ret:
        break
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #Resize the frame to the input shape of the model
    resized = cv2.resize(gray, (input_shape[1], input_shape[0]))
    
    # Normalize the pixel values
    normalized = resized.astype('float32') / 255.0
    
    # Add a batch dimension to the input tensor
    tensor = np.expand_dims(normalized, axis=0)
    
    # Make a prediction using the model
    prediction = model.predict(tensor)[0]
    
    # Add the predicted headcount to the total headcount
    headcount += prediction
    
# Release the video file and print the total headcount
cap.release()
print('Total headcount:', int(sum(headcount)))

1/1 [==============================] - 0s 24ms/step
Total headcount: 0
